# Hartree-Fock Analytic Nuclear Gradients Implementation

This notebook demonstrates how to compute analytic nuclear gradients for Hartree-Fock theory using Psi4Numpy.
The implementation follows the equations from Szabo and Ostlund's "Modern Quantum Chemistry".

## Theory Overview
For a closed-shell system, the Hartree-Fock energy gradient with respect to nuclear coordinate x is:

$$\frac{\partial E}{\partial x_i} = \frac{\partial E_{nuc}}{\partial x_i} + \sum_{\mu\nu} D_{\mu\nu} \frac{\partial h_{\mu\nu}}{\partial x_i} + \frac{1}{2} \sum_{\mu\nu\lambda\sigma} D_{\mu\nu} D_{\lambda\sigma} \frac{\partial (\mu\nu|\lambda\sigma)}{\partial x_i} - \sum_{\mu\nu} W_{\mu\nu} \frac{\partial S_{\mu\nu}}{\partial x_i}$$

equivalently,

$$ \frac{\partial E}{\partial x_i} = \frac{\partial E_{nuc}}{\partial x_i} + \sum_{\mu\nu} D_{\mu\nu} \frac{\partial h_{\mu\nu}}{\partial x_i} + \frac{1}{2} \sum_{\mu\nu\lambda\sigma} D_{\mu\nu} D_{\lambda\sigma} \frac{\partial (\mu\nu|\lambda\sigma)}{\partial x_i}- \sum_{pq} F_{pq} \frac{\partial S_{pq}}{\partial R_A} $$
and similarly for the $y$ and $z$ coordinates.  Importantly, each atom has $x$, $y$, and $z$ coordinates so this yields a gradient
vector with $3 N_{\rm atoms}$ elements.

Where:
- $E_{nuc}$ is the nuclear repulsion energy
- $D_{\mu\nu}$ is the density matrix 
- $h_{\mu\nu}$ is the one-electron integral matrix in the AO basis
- $(\mu\nu|\lambda\sigma)$ are the two-electron integrals in the AO basis
- $S_{\mu\nu}$ is the overlap matrix in the AO basis
- $W_{\mu\nu}$ is the energy-weighted density matrix
- $F_{p q}$ is the Fock matrix in the MO basis
- $S_{pq}$ is the overlap matrix in the MO basis

# Numerical Hartree-Fock gradients
The Hartree-Fock energy gradient may also be approximated using centered finite differences as follows:
$$\frac{\partial E}{\partial x_i} \approx \frac{ E(x_1, ..., x_i + \delta x, x_{i+1}, ... x_N) - E(x_1, ..., x_i - \delta x, x_{i+1}, ... x_N) }{2 \delta x} $$
where $\delta x$ is a sufficiently small step along the nuclear coordinates.  

**Importantly** Numerical gradients in this approximation require two full solutions to the RHF energy for each nuclear degree of freedom, so in total $6 N_{atoms}$ RHF calculations are needed for each numerical gradient evaluation.  The analytical gradients require only a single solution of the RHF energy and then a series of contractions using the existing RHF density matrix.  

Nevertheless, we can set up a cell to compute the numerical gradient as a reference.

In [1]:
# First import the necessary libraries
import psi4
import numpy as np
np.set_printoptions(precision=15, linewidth=200, suppress=True)

In [2]:
_expected_nuclear_gradient = np.array([
    [0.00000000000000,  0.00000000000000,  2.99204046891092],
    [0.00000000000000, -2.05144597283373, -1.49602023445546],
    [0.00000000000000,  2.05144597283373, -1.49602023445546],
])

_overlap_gradient = np.array([
    [-0.00000000000000, -0.00000000000000,  0.30728746121587],
    [ 0.00000000000000, -0.14977126575800, -0.15364373060793],
    [-0.00000000000000,  0.14977126575800, -0.15364373060793],
])

_potential_gradient = np.array([
    [-0.00000000000000,  0.00000000000002, -6.81982772856799],
    [-0.00000000000000,  4.38321774316664,  3.40991386428399],
    [ 0.00000000000000, -4.38321774316666,  3.40991386428400],
])
_kinetic_gradient = np.array([
    [ 0.00000000000000, -0.00000000000000,  0.66968290617933],
    [ 0.00000000000000, -0.43735698924315, -0.33484145308966],
    [-0.00000000000000,  0.43735698924315, -0.33484145308967],
])

_coulomb_gradient = np.array([
    [ 0.00000000000000, -0.00000000000002,  3.34742251141627],
    [ 0.00000000000000, -2.03756324433539, -1.67371125570813],
    [-0.00000000000000,  2.03756324433541, -1.67371125570814],
])

_exchange_gradient = np.array([
    [-0.00000000000000,  0.00000000000000, -0.43559674130726],
    [-0.00000000000000,  0.26932748463493,  0.21779837065363],
    [ 0.00000000000000, -0.26932748463493,  0.21779837065363],
])

In [3]:
def modify_geometry_string(geometry_string, displacement_array):
    """
    Extracts Cartesian coordinates from a Psi4 geometry string, applies a
    transformation function to the coordinates, and returns a new geometry string.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        transformation_function (callable): A function that takes a NumPy
            array of Cartesian coordinates (N x 3) as input and returns a
            NumPy array of the same shape with the transformed coordinates.

    Returns:
        str: A new Psi4 molecular geometry string with the transformed coordinates.
    """
    lines = geometry_string.strip().split('\n')
    atom_data = []
    symmetry = None

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.lower().startswith("symmetry"):
            symmetry = line
            continue
        parts = line.split()
        if len(parts) == 4:
            atom = parts[0]
            try:
                x, y, z = map(float, parts[1:])
                atom_data.append([atom, x, y, z])
            except ValueError:
                # Handle cases where the line might not be atom coordinates
                pass

    if not atom_data:
        return ""

    coordinates = np.array([[data[1], data[2], data[3]] for data in atom_data])

    # Apply the transformation function
    transformed_coordinates = displacement_array  + coordinates

    new_geometry_lines = []
    for i, data in enumerate(atom_data):
        atom = data[0]
        new_geometry_lines.append(f"{atom} {transformed_coordinates[i, 0]:.8f} {transformed_coordinates[i, 1]:.8f} {transformed_coordinates[i, 2]:.8f}")

    new_geometry_string = "\n".join(new_geometry_lines)
    if symmetry:
        new_geometry_string += f"\n{symmetry}"

    return f"""{new_geometry_string}"""


def run_psi4_calculation(geometry_string, displacement_array, basis_set='sto-3g', method='scf'):
    """
    Runs a Psi4 calculation with the given geometry string, basis set, and method.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        displacement_array (np.ndarray): An array of displacements to apply to the coordinates.
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.
        

    Returns:
        dict: A dictionary containing the results of the Psi4 calculation.
    """
    # Modify the geometry string with the displacement array
    modified_geometry_string = modify_geometry_string(geometry_string, displacement_array)

    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # Run the Psi4 calculation
    try:
        psi4.geometry(modified_geometry_string)
        energy = psi4.energy(method)
    except Exception as e:
        print(f"Error during Psi4 calculation: {e}")
        return None

    return energy

def compute_psi4_analytical_gradient(geometry_string, basis_set='sto-3g', method='scf'):
    """
    Computes the energy gradient for a given geometry string using Psi4.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.

    Returns:
        np.ndarray: The energy gradient as a NumPy array.
    """
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # Run the Psi4 calculation
    try:
        psi4.geometry(geometry_string)
        gradient = psi4.gradient(method)
    except Exception as e:
        print(f"Error during Psi4 calculation: {e}")
        return None

    return np.asarray(gradient)

def compute_psi4_numerical_gradient(geometry_string, displacement_unit=0.01, basis_set='sto-3g', method='scf'):
    """
    NEEDS COMPLETING: Computes the numerical gradient for a given geometry string using Psi4.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        displacement_unit (float): The unit of displacement for numerical gradient calculation, defaults to 0.01.
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.

    Returns:
        np.ndarray: The numerical gradient as a NumPy array.
    """
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # Get the number of atoms from the geometry string
    num_atoms = len(geometry_string.strip().split('\n')) - 1

    # Initialize the numerical gradient array
    numerical_gradient = np.zeros((num_atoms, 3))

    # loop over all atoms
    for i in range(num_atoms):

        # loop over all three dimensions
        for j in range(3):
            # Create a displacement unit vector
            displacement_array = np.zeros((num_atoms, 3))
            displacement_array[i, j] = displacement_unit
            
            # print the displacement unit vector
            print(displacement_array)

            # Insert code to compute psi4 energy at forward displacement
            e_f = run_psi4_calculation(geometry_string, displacement_array, basis_set, method)

            # Insert code to compute psi4 energy at backwards displacement
            displacement_array *= -1
            e_b = run_psi4_calculation(geometry_string, displacement_array, basis_set, method)

            # Insert code to compute finite difference along this displacement
            grad_element = (e_f - e_b) / (2 * 1.88973 * displacement_unit)

            # Insert code to store this to the appropriate gradient element
            numerical_gradient[i, j] = grad_element


    # return the gradient
    return numerical_gradient 


 


In [4]:
starting_string = """
O 0.0 0.0 0.0
H 0.0 0.757 0.587
H 0.0 -0.757 0.587
symmetry c1
"""

starting_displacement = np.array([[0, 0.0, 0.0], [0, 0, 0,], [0, 0, .0]])

# Run the Psi4 calculation
energy = run_psi4_calculation(starting_string, starting_displacement, basis_set='sto-3g', method='scf')
gradient = compute_psi4_analytical_gradient(starting_string, basis_set='sto-3g', method='scf')

# test the numerical gradient
numerical_gradient = compute_psi4_numerical_gradient(starting_string, displacement_unit=0.01, basis_set='sto-3g', method='scf')


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Sat Apr 26 19:22:56 2025

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line    81 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 
    atoms 2-3 entry H          line    19 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         -------------------------------------------------

In [5]:
print("Energy:", energy)
print("Gradient:\n", gradient)
print("Numerical Gradient:\n", numerical_gradient)

# compute error between numerical and analytical gradient
error = numerical_gradient - gradient
print("Error:\n", error)
print("Norm of error:\n",np.linalg.norm(error))

Energy: -74.96306312979318
Gradient:
 [[ 0.                 0.000000000000001  0.061008877848977]
 [-0.                -0.023592244369028 -0.030504438924483]
 [ 0.                 0.023592244369025 -0.030504438924483]]
Numerical Gradient:
 [[ 0.                -0.000000000000752  0.061016771477982]
 [ 0.000000000000752 -0.023644674278839 -0.030509434267519]
 [ 0.                 0.023644674272823 -0.030509434272031]]
Error:
 [[-0.                -0.000000000000753  0.000007893629005]
 [ 0.000000000000752 -0.000052429909811 -0.000004995343036]
 [-0.                 0.000052429903798 -0.000004995347548]]
Norm of error:
 7.489997719072365e-05


# Function to compute nuclear gradient
$$\frac{\partial E_{nuc}}{\partial x_i}$$ 


In [6]:
def compute_nuclear_repulsion_gradient(geometry_string):
    """
    Method to compute the nuclear repulsion gradient

    Arguments
    ---------
    geometry_string : str
        psi4 molecule string

    The nuclear repulsion gradient only depends on the atom identities and positions
    """
    # Define your molecular geometry
    molecule = psi4.geometry(geometry_string)

    # get the nuclear repulsion gradient
    nuclear_repulsion_gradient = np.asarray(molecule.nuclear_repulsion_energy_deriv1())
    
    return nuclear_repulsion_gradient

# Compute the nuclear repulsion gradient using the starting_string
nuclear_repulsion_gradient = compute_nuclear_repulsion_gradient(starting_string)
print("Nuclear Repulsion Gradient:\n", nuclear_repulsion_gradient)


# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(nuclear_repulsion_gradient, _expected_nuclear_gradient):
    print("Nuclear repulsion gradient is correct.")
else:
    print("Nuclear repulsion gradient is incorrect.")
    print("Computed:\n", nuclear_repulsion_gradient)
    print("Expected:\n", _expected_nuclear_gradient)

Nuclear Repulsion Gradient:
 [[ 0.                 0.                 2.992040468910919]
 [ 0.                -2.051445972833726 -1.49602023445546 ]
 [ 0.                 2.051445972833726 -1.49602023445546 ]]
Nuclear repulsion gradient is correct.


# Function to compute Fock matrix expressed in the MO basis
$$ F_{ij} = V_{ij} + T_{ij} + 2 \sum_{k}^{occ} (ii|kk) - \sum_{k}^{occ} (ik|kj) $$


In [7]:
def compute_fock_matrix_term(geometry_string, basis_set='sto-3g', method='scf'):
    """
    Method to compute the Fock matrix

    Arguments
    ---------

    geometry_string : str
        psi4 molecule string

    basis_set : str
        basis set to use for the calculation, defaults to 'sto-3g'

    method : str
        quantum chemistry method to use for the calculation, defaults to 'scf'

    The Fock matrix is the matrix representation of the Fock operator, which is used in Hartree-Fock calculations.
    To compute the Fock matrix in the MO basis, we need the one-electron and two-electron integrals and the density matrix.
    We will get these from a converged Hartree-Fock calculation.
    """

    # set up the molecule
    molecule = psi4.geometry(geometry_string)

    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # set up the geometry
    psi4.geometry(geometry_string)

    # run the Hartree-Fock calculation
    rhf_e, wfn = psi4.energy(method, return_wfn=True)

    # get number of atoms
    n_atoms = molecule.natom()

    # get the number of orbitals and the number of doubly occupied orbitals
    n_orbitals = wfn.nmo()
    n_docc = wfn.nalpha()

    # get the orbital transformation matrix
    C = wfn.Ca() # -> as psi4 matrix object
    Cnp = np.asarray(C) # -> as numpy array


    # instantiate the MintsHelper object
    mints = psi4.core.MintsHelper(wfn.basisset())

    # get the one-electron integrals
    H_ao = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential())

    # transform H_ao to the MO basis
    H_mo = np.einsum('uj, vi, uv', Cnp, Cnp, H_ao)

    # get the two-electron integrals, use psi4 to transform into the MO basis because that is more efficient
    ERI =  np.asarray(mints.mo_eri(C, C, C, C))

    # Build the Fock matrix
    F = H_mo + 2 * np.einsum("ijkk->ij", ERI[:, :, :n_docc, :n_docc]) 
    F -= np.einsum("ikkj->ij", ERI[:, :n_docc, :n_docc, :] )

    # now compute the overlap gradient and contract with Fock matrix
    overlap_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    overlap_gradient = np.zeros(3 * n_atoms)
    for atom_index in range(n_atoms):

        # Derivatives with respect to x, y, and z of the current atom
        for cart_index in range(3):
            deriv_index = 3 * atom_index + cart_index
            # Get overlap derivatives for this atom and Cartesian component
            overlap_derivs[deriv_index, :, :] = np.asarray(mints.mo_oei_deriv1("OVERLAP", atom_index, C, C)[cart_index])
            overlap_gradient[deriv_index] = -2.0 * np.einsum('ii,ii->', F[:n_docc, :n_docc], overlap_derivs[deriv_index, :n_docc, :n_docc])

    return overlap_gradient

In [14]:
def compute_one_electron_integral_gradient_terms(geometry_string, basis_set='sto-3g', method='scf'):
    """
    NEEDS COMPLETING: Method to compute the one-electron integral gradient terms

    Arguments
    ---------
    geometry_string : str
        psi4 molecule string

    basis_set : str
        basis set to use for the calculation, defaults to 'sto-3g'

    method : str
        quantum chemistry method to use for the calculation, defaults to 'scf'

    The one-electron integral gradient terms are the derivatives of the one-electron integrals with respect to the nuclear coordinates.
    To compute the one-electron integral gradient terms, we need the one-electron integrals and the nuclear repulsion gradient.
    We will get these from a converged Hartree-Fock calculation.
    """
    # set up the molecule
    molecule = psi4.geometry(geometry_string)
    
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # set up the geometry
    psi4.geometry(geometry_string)

    # run the Hartree-Fock calculation
    rhf_e, wfn = psi4.energy(method, return_wfn=True)

    # get the number of orbitals and the number of doubly occupied orbitals
    n_orbitals = wfn.nmo()
    n_docc = wfn.nalpha()

    # get the number of atoms
    n_atoms = molecule.natom()

    # get the orbital transformation matrix
    C = wfn.Ca() # -> as psi4 matrix object
    Cnp = np.asarray(C) # -> as numpy array

    # get the Density matrix by summing over the occupied orbital transformation matrix
    Cocc = Cnp[:, :n_docc]
    D = np.einsum("pi,qi->pq", Cocc, Cocc) * 2 # [Szabo:1996] Eqn. 3.145, pp. 139

    # instantiate the MintsHelper object
    mints = psi4.core.MintsHelper(wfn.basisset())

    # initialize the one-electron integrals derivative matrices
    kinetic_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    potential_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    kinetic_gradient = np.zeros(3 * n_atoms)
    potential_gradient = np.zeros(3 * n_atoms)


    # loop over all of the atoms
    for atom_index in range(n_atoms):
        # Derivatives with respect to x, y, and z of the current atom
         for cart_index in range(3):
            deriv_index = 3 * atom_index + cart_index

            # get the one-electron integral derivatives
            kinetic_derivs[deriv_index] = np.asarray(mints.ao_oei_deriv1("KINETIC", atom_index)[cart_index])
            potential_derivs[deriv_index] = np.asarray(mints.ao_oei_deriv1("POTENTIAL", atom_index)[cart_index])

            # add code to contract kinetic_derivs with D
            kinetic_gradient[deriv_index] = np.einsum("uv,uv->", D, kinetic_derivs[deriv_index, :, :])

            # add code to contract potential_derivs with D
            potential_gradient[deriv_index] = np.einsum("uv,uv->", D, potential_derivs[deriv_index, :, :])

    
    # add code to return the kinet_gradient and potential_gradient
    return kinetic_gradient, potential_gradient

In [15]:
# compute the overlap gradient ter
overlap_gradient = compute_fock_matrix_term(starting_string)

# compute the one-electron integral gradient terms
kinetic_gradient, potential_gradient = compute_one_electron_integral_gradient_terms(starting_string)




Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Sat Apr 26 19:27:28 2025

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line    81 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 
    atoms 2-3 entry H          line    19 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         -------------------------------------------------

In [17]:
# check overlap gradient against expected
print("Expected overlap gradient:\n", _overlap_gradient)
print("Computed overlap gradient:\n", overlap_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(overlap_gradient.reshape(3,3), _overlap_gradient):
    print("Overlap gradient is correct.")
else:
    print("Overlap gradient is incorrect.")

# check kinetic gradient against expected
print("Expected kinetic gradient:\n", _kinetic_gradient)
print("Computed kinetic gradient:\n", kinetic_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(kinetic_gradient.reshape(3,3), _kinetic_gradient):
    print("Kinetic gradient is correct.")
else:
    print("Kinetic gradient is incorrect.")

# check potential gradient against expected
print("Expected potential gradient:\n", _potential_gradient)
print("Computed potential gradient:\n", potential_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(potential_gradient.reshape(3,3), _potential_gradient):
    print("Potential gradient is correct.")
else:
    print("Potential gradient is incorrect.")


Expected overlap gradient:
 [[-0.               -0.                0.30728746121587]
 [ 0.               -0.149771265758   -0.15364373060793]
 [-0.                0.149771265758   -0.15364373060793]]
Computed overlap gradient:
 [[-0.                -0.000000000000003  0.307287461214727]
 [ 0.                -0.149771265757692 -0.153643730607357]
 [ 0.                 0.149771265757695 -0.153643730607359]]
Overlap gradient is correct.
Expected kinetic gradient:
 [[ 0.               -0.                0.66968290617933]
 [ 0.               -0.43735698924315 -0.33484145308966]
 [-0.                0.43735698924315 -0.33484145308967]]
Computed kinetic gradient:
 [[-0.                -0.000000000000002  0.669682906178145]
 [ 0.                -0.437356989243085 -0.334841453089072]
 [-0.                 0.437356989243087 -0.334841453089073]]
Kinetic gradient is correct.
Expected potential gradient:
 [[-0.                0.00000000000002 -6.81982772856799]
 [-0.                4.38321774316664

In [24]:
def compute_two_electron_integral_gradient_terms(geometry_string, basis_set='sto-3g', method='scf'):
    """
    NEEDS COMPLETING: Method to compute the two-electron integral gradient terms

    Arguments
    ---------
    geometry_string : str
        psi4 molecule string

    basis_set : str
        basis set to use for the calculation, defaults to 'sto-3g'

    method : str
        quantum chemistry method to use for the calculation, defaults to 'scf'

    The two-electron integral gradient terms are the derivatives of the two-electron integrals with respect to the nuclear coordinates.
    To compute the two-electron integral gradient terms, we need the two-electron integrals and the nuclear repulsion gradient.
    We will get these from a converged Hartree-Fock calculation.
    """
    # set up the molecule
    molecule = psi4.geometry(geometry_string)
    
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })
    
    # set up the geometry
    psi4.geometry(geometry_string)

    # run the Hartree-Fock calculation
    rhf_e, wfn = psi4.energy(method, return_wfn=True)

    # get the number of orbitals and the number of doubly occupied orbitals
    n_orbitals = wfn.nmo()
    n_docc = wfn.nalpha()

    # get the number of atoms
    n_atoms = molecule.natom()

    # get the orbital transformation matrix
    C = wfn.Ca() # -> as psi4 matrix object
    Cnp = np.asarray(C) # -> as numpy array

    # get the Density matrix by summing over the occupied orbital transformation matrix
    Cocc = Cnp[:, :n_docc]
    D = np.einsum("pi,qi->pq", Cocc, Cocc)  # [Szabo:1996] Eqn. 3.145, pp. 139

    # instantiate the MintsHelper object
    mints = psi4.core.MintsHelper(wfn.basisset())

    # initialize the two-electron integrals derivative matrices
    eri_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals, n_orbitals, n_orbitals))
    J_deriv = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    K_deriv = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    J_gradient = np.zeros(3 * n_atoms)
    K_gradient = np.zeros(3 * n_atoms)

    # loop over all of the atoms
    for atom_index in range(n_atoms):
        # Derivatives with respect to x, y, and z of the current atom
        for cart_index in range(3):
            deriv_index = 3 * atom_index + cart_index

            # get the two-electron integral derivatives
            eri_derivs[deriv_index] = np.asarray(mints.ao_tei_deriv1(atom_index)[cart_index])

            # add code to contract eri_derivs with D to get J_deriv. J_uv = 2 * sum_ls (uv|ls) D_ls 
            J_deriv[deriv_index] = 2 * np.einsum("uvls,ls->uv", eri_derivs[deriv_index, :, :, :, :], D)

            # add code to contract eri_derivs with D to get K_deriv. K_uv = -1 * sum_ls (ul|vs) D_ls
            K_deriv[deriv_index] = -1 * np.einsum("ulvs,ls->uv", eri_derivs[deriv_index, :, :, :, :], D)

            # add code to contract J_deriv with D to get J_gradient
            J_gradient[deriv_index] = np.einsum("uv,uv->", D, J_deriv[deriv_index, :, :])

            # add code to contract K_deriv with D to get K_gradient
            K_gradient[deriv_index] = np.einsum("uv,uv->", D, K_deriv[deriv_index, :, :])

    # add code to return the J_gradient and K_gradient
    return J_gradient, K_gradient

In [25]:
J_gradient, K_gradient = compute_two_electron_integral_gradient_terms(starting_string)



Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Sun Apr 27 07:41:12 2025

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line    81 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 
    atoms 2-3 entry H          line    19 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         -------------------------------------------------

In [26]:
# check J gradient against expected
print("Expected J gradient:\n", _coulomb_gradient)
print("Computed J gradient:\n", J_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(J_gradient.reshape(3,3), _coulomb_gradient):
    print("J gradient is correct.")
else:   
    print("J gradient is incorrect.")
# check K gradient against expected
print("Expected K gradient:\n", _exchange_gradient)
print("Computed K gradient:\n", K_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(K_gradient.reshape(3,3), _exchange_gradient):
    print("K gradient is correct.")
else:
    print("K gradient is incorrect.")


Expected J gradient:
 [[ 0.               -0.00000000000002  3.34742251141627]
 [ 0.               -2.03756324433539 -1.67371125570813]
 [-0.                2.03756324433541 -1.67371125570814]]
Computed J gradient:
 [[-0.000000000000002 -0.000000000000023  3.347422511402989]
 [ 0.000000000000002 -2.037563244333946 -1.673711255701491]
 [-0.                 2.037563244333969 -1.673711255701498]]
J gradient is correct.
Expected K gradient:
 [[-0.                0.               -0.43559674130726]
 [-0.                0.26932748463493  0.21779837065363]
 [ 0.               -0.26932748463493  0.21779837065363]]
Computed K gradient:
 [[ 0.                 0.000000000000004 -0.435596741305196]
 [-0.                 0.26932748463448   0.217798370652597]
 [ 0.                -0.269327484634483  0.217798370652599]]
K gradient is correct.


In [29]:
# compute the total energy gradient
total_energy_gradient = nuclear_repulsion_gradient.flatten() + kinetic_gradient + potential_gradient + J_gradient + K_gradient + overlap_gradient
print("Total energy gradient:\n", total_energy_gradient.reshape(3,3))
# Compare the computed total energy gradient with the expected value psi4
if np.allclose(total_energy_gradient.reshape(3,3), gradient):
    print("Total energy gradient is correct.")
else:  
    print("Total energy gradient is incorrect.")
    print("Computed:\n", total_energy_gradient.reshape(3,3))
    print("Expected:\n", gradient)

Total energy gradient:
 [[ 0.                 0.000000000000003  0.061008877848787]
 [-0.                -0.023592244369113 -0.030504438924387]
 [ 0.                 0.023592244369111 -0.030504438924387]]
Total energy gradient is correct.
